# Исследование надежности заемщиков.

## Введение

Заказчик — кредитный отдел банка. Перед нами стоит задача разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок.
Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.


## Обзор данных

In [39]:
import pandas as pd
import numpy as np

In [40]:
data = pd.read_csv('/datasets/data.csv')

In [41]:
data.info()
data.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Названия столбцов в корректном формате. 

## Предобработка данных

### Заполнение пропусков

In [42]:
#data.isna().sum()
#data[data['days_employed'].isna()].head(10)
#data[data['total_income'].isna()].head(10)
#days_employed_mediana = data['days_employed'].median()
#total_income_mediana = data['total_income'].median()
#data['days_employed'].fillna(days_employed_mediana,inplace=True)
#data['total_income'].fillna(total_income_mediana,inplace=True)
#data.info()
#data.head(10)

**Сгрупируем стаж по образованию**

In [43]:
data['education'] = data['education'].str.lower()
data.groupby('education')['days_employed'].count()
days_employed_group_mean = data.groupby('education')['days_employed'].mean()
print(days_employed_group_mean)

days_employed_group_median = data.groupby('education')['days_employed'].median()
print(days_employed_group_median)

education
высшее                  38323.055702
начальное              127842.088750
неоконченное высшее     17692.508357
среднее                 72538.686698
ученая степень         116630.048157
Name: days_employed, dtype: float64
education
высшее                -1342.373432
начальное              -551.062561
неоконченное высшее   -1046.437583
среднее               -1184.327177
ученая степень        -1380.316041
Name: days_employed, dtype: float64


Медианные и средние значения выглядят неправдоподобно

**Сгруппируем зарплату по образованию**

In [44]:
data.groupby('education')['total_income'].count()
total_income_group_mean = data.groupby('education')['total_income'].mean()
print(total_income_group_mean)

total_income_group_median = data.groupby('education')['total_income'].median()
print(total_income_group_median)


education
высшее                 207142.515219
начальное              132155.513626
неоконченное высшее    181534.022774
среднее                153715.643971
ученая степень         174750.155792
Name: total_income, dtype: float64
education
высшее                 175340.818855
начальное              117137.352825
неоконченное высшее    160115.398644
среднее                136478.643244
ученая степень         157259.898555
Name: total_income, dtype: float64


Медианные и средние значения, также выглядят неправдоподобно(с неоконченным высшим и высшим образованием получают больше чем люди с учёной степенью).

**Сгруппируем стаж по возрасту**

In [45]:
data.groupby('dob_years')['days_employed'].count()
days_employed_group_mean = data.groupby('dob_years')['days_employed'].mean()
print(days_employed_group_mean)

days_employed_group_median = data.groupby('dob_years')['days_employed'].median()
print(days_employed_group_median)

dob_years
0      65937.471974
19      -633.678086
20      -684.944308
21      -709.440930
22      1239.982481
23      -827.309437
24     -1026.405485
25     -1088.406453
26       826.912862
27      1027.345107
28      -609.021658
29     -1553.823200
30     -1696.039355
31      -967.511501
32       427.651495
33      -481.794213
34       196.083153
35     -1454.909683
36       -21.296136
37      1566.413139
38      3095.214328
39       501.209953
40      2446.420996
41      1586.992875
42      3393.631090
43      4530.036926
44      4435.213337
45      6302.805072
46      8561.545242
47      8479.909716
48     12588.914967
49     20891.619193
50     45116.098565
51     63444.075862
52     77716.268854
53     89171.803901
54    122836.034309
55    148044.355465
56    153655.238208
57    190422.749553
58    185391.153289
59    224555.447555
60    260794.805996
61    244109.801522
62    272857.075474
63    291629.371951
64    285931.680485
65    285675.504221
66    312465.270899
67    3167

Нереальные значения, взаимосвязь не прослеживается

**Сгруппируем зарплату по возрасту**

In [46]:
data.groupby('dob_years')['total_income'].count()
total_income_group_mean = data.groupby('dob_years')['total_income'].mean()
print(total_income_group_mean)
total_income_group_median = data.groupby('dob_years')['total_income'].median()
print(total_income_group_median)

dob_years
0     158337.955712
19    106212.141123
20    126993.295912
21    143629.769631
22    139840.678505
23    138463.153535
24    150446.100901
25    157572.291387
26    157955.848949
27    177110.490144
28    164685.071256
29    169246.438988
30    171970.318639
31    173887.307980
32    167407.477166
33    181604.680340
34    178092.098754
35    177914.045031
36    182428.883307
37    183334.890130
38    168196.928236
39    184250.715208
40    176353.724146
41    177454.879365
42    184710.960430
43    184750.762352
44    177817.361512
45    173600.360807
46    186505.550986
47    169281.635880
48    177231.169491
49    176097.353303
50    165649.381950
51    170219.301449
52    168032.526480
53    167540.434260
54    156215.871072
55    155247.573839
56    161652.131498
57    155360.539014
58    157405.443193
59    154988.350033
60    142604.267812
61    151922.056389
62    148841.527150
63    143097.516441
64    161777.457555
65    151717.488838
66    131718.850062
67    1379

И медианные и средние значения похожи на правду. Зарплата увеличивается до пика работоспособного возраста ~ 50 лет. Позже заменим пропуски в 'total_income' медианой.

Столбцы days_employed и total_income имеют одинаковое значение - "non-null" (=19351). В столбце days_employed встречаются неправдоподобные значения.

In [47]:
print(f'Доля пропущеных значений = {(21525-19351)/21525:,.2%}')

Доля пропущеных значений = 10.10%


Возможные причины появления пропусков:
- Человеческий фактор
- Технологический

Судя по одинаковому количеству пропусков, между ними прямая взаимосвязь(возможно это студенты).
Заполнение пропусков медианным значением для количественных переменных наиболее точно отражает действительность.

In [48]:
print(data[data['education'] == 'неоконченное высшее'].count())
print(f'Доля студентов в пропущеных значениях = {675/(21525-19351):,.2%}')

children            744
days_employed       675
dob_years           744
education           744
education_id        744
family_status       744
family_status_id    744
gender              744
income_type         744
debt                744
total_income        675
purpose             744
dtype: int64
Доля студентов в пропущеных значениях = 31.05%


Треть заёмщиков являются студентами или не закончили ВУЗ

### Шаг 2.2 Проверка данных на аномалии и исправления.

**Столбец 'children'**

In [49]:
print(data['children'].value_counts())

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64


Реалистичные значения в семьях без детей и от 1 до 3. 76 семей с 20 детьми(маловероятно) и 47 с "-1" ребёнком(скорее всего "-" это дефис, заменим на "1"). 123 значения - незначительно(<1%).

In [50]:
display(data[data['children'] == 20].head(10))
display(data[data['children']== -1].head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
606,20,-880.221113,21,среднее,1,женат / замужем,0,M,компаньон,0,145334.865002,покупка жилья
720,20,-855.595512,44,среднее,1,женат / замужем,0,F,компаньон,0,112998.738649,покупка недвижимости
1074,20,-3310.411598,56,среднее,1,женат / замужем,0,F,сотрудник,1,229518.537004,получение образования
2510,20,-2714.161249,59,высшее,0,вдовец / вдова,2,F,сотрудник,0,264474.835577,операции с коммерческой недвижимостью
2941,20,-2161.591519,0,среднее,1,женат / замужем,0,F,сотрудник,0,199739.941398,на покупку автомобиля
3302,20,NaN,35,среднее,1,Не женат / не замужем,4,F,госслужащий,0,NaN,профильное образование
3396,20,NaN,56,высшее,0,женат / замужем,0,F,компаньон,0,NaN,высшее образование
3671,20,-913.161503,23,среднее,1,Не женат / не замужем,4,F,сотрудник,0,101255.492076,на покупку подержанного автомобиля
3697,20,-2907.910616,40,среднее,1,гражданский брак,1,M,сотрудник,0,115380.694664,на покупку подержанного автомобиля
3735,20,-805.044438,26,высшее,0,Не женат / не замужем,4,M,сотрудник,0,137200.646181,ремонт жилью


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
291,-1,-4417.703588,46,среднее,1,гражданский брак,1,F,сотрудник,0,102816.346412,профильное образование
705,-1,-902.084528,50,среднее,1,женат / замужем,0,F,госслужащий,0,137882.899271,приобретение автомобиля
742,-1,-3174.456205,57,среднее,1,женат / замужем,0,F,сотрудник,0,64268.044444,дополнительное образование
800,-1,349987.852217,54,среднее,1,Не женат / не замужем,4,F,пенсионер,0,86293.724153,дополнительное образование
941,-1,NaN,57,среднее,1,женат / замужем,0,F,пенсионер,0,NaN,на покупку своего автомобиля
1363,-1,-1195.264956,55,среднее,1,женат / замужем,0,F,компаньон,0,69550.699692,профильное образование
1929,-1,-1461.303336,38,среднее,1,Не женат / не замужем,4,M,сотрудник,0,109121.569013,покупка жилья
2073,-1,-2539.761232,42,среднее,1,в разводе,3,F,компаньон,0,162638.609373,покупка жилья
3814,-1,-3045.290443,26,среднее,1,гражданский брак,1,F,госслужащий,0,131892.785435,на проведение свадьбы
4201,-1,-901.101738,41,среднее,1,женат / замужем,0,F,госслужащий,0,226375.766751,операции со своей недвижимостью


In [51]:
index_1 = data[data['children']==-1].index 

index_20 = data[data['children']==20].index

str_del_count=0

for drop_str in index_1:
        data = data.drop(drop_str)
        str_del_count+=1

for drop_str in index_20:
        data = data.drop(drop_str)
        str_del_count+=1

data = data.reset_index(drop=True)
print(data['children'].value_counts())
print(data.info())         

0    14149
1     4818
2     2055
3      330
4       41
5        9
Name: children, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21402 entries, 0 to 21401
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21402 non-null  int64  
 1   days_employed     19240 non-null  float64
 2   dob_years         21402 non-null  int64  
 3   education         21402 non-null  object 
 4   education_id      21402 non-null  int64  
 5   family_status     21402 non-null  object 
 6   family_status_id  21402 non-null  int64  
 7   gender            21402 non-null  object 
 8   income_type       21402 non-null  object 
 9   debt              21402 non-null  int64  
 10  total_income      19240 non-null  float64
 11  purpose           21402 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None


**Столбец 'days_employed'**

In [52]:
print(data['days_employed'])
print('Отрицательных значений:',data[data['days_employed']<0]['days_employed'].count())

0         -8437.673028
1         -4024.803754
2         -5623.422610
3         -4124.747207
4        340266.072047
             ...      
21397     -4529.316663
21398    343937.404131
21399     -2113.346888
21400     -3112.481705
21401     -1984.507589
Name: days_employed, Length: 21402, dtype: float64
Отрицательных значений: 15809


Присутствуют слишком большие значения (~ 340266.072047), возможно значение в часах, но настораживает количество цифр после запятой. В отрицательных значениях, возможно "-" это дефис. Поскольку уточнить информацию невозможно, а аномалий большинство(18080-отрицательных значений и 2174-пустых), и значения столбца не влияют на выполнение задачи, удаляем его.  

In [53]:
data.drop('days_employed', axis='columns', inplace=True)
print(data.columns)

Index(['children', 'dob_years', 'education', 'education_id', 'family_status',
       'family_status_id', 'gender', 'income_type', 'debt', 'total_income',
       'purpose'],
      dtype='object')


**Столбец 'dob_years'**

In [54]:
data['dob_years'].value_counts().sort_index()

0     100
19     14
20     51
21    110
22    183
23    252
24    263
25    356
26    406
27    490
28    501
29    543
30    536
31    556
32    506
33    577
34    597
35    614
36    553
37    531
38    595
39    572
40    603
41    603
42    592
43    510
44    543
45    494
46    469
47    480
48    536
49    505
50    509
51    446
52    483
53    457
54    476
55    441
56    482
57    457
58    461
59    441
60    376
61    353
62    351
63    268
64    263
65    194
66    183
67    167
68     99
69     83
70     65
71     58
72     33
73      8
74      6
75      1
Name: dob_years, dtype: int64

Основные заёмщики в возрасте от 19 до 75 лет, что приближено к реалиям. Аномалия - 101 заёмщик, которому 0 лет(<0.5%, при невозможности уточнения можно удалить)

In [55]:
data['dob_years'] = data.query('dob_years > 0')

**Столбец 'education'**

In [56]:
data['education'].value_counts()

среднее                15136
высшее                  5237
неоконченное высшее      741
начальное                282
ученая степень             6
Name: education, dtype: int64

Корректный столбец

**Столбец 'education_id '**

In [57]:
data['education'].value_counts(), data['education_id'].value_counts()

(среднее                15136
 высшее                  5237
 неоконченное высшее      741
 начальное                282
 ученая степень             6
 Name: education, dtype: int64,
 1    15136
 0     5237
 2      741
 3      282
 4        6
 Name: education_id, dtype: int64)

5 видов образования.

**Столбец 'family_status'**   **Столбец 'family_status_id'**

In [58]:
data['family_status'].value_counts()
data['family_status_id'].value_counts()

0    12302
1     4160
4     2799
3     1189
2      952
Name: family_status_id, dtype: int64

Значения в столбцах совпадают. Данные столбца 'family_status'(Не женат / не замужем) надо привести к одному регистру. 

In [59]:
data['family_status'] = data['family_status'].str.lower()
data['family_status'].value_counts()
data['family_status_id'].value_counts()

0    12302
1     4160
4     2799
3     1189
2      952
Name: family_status_id, dtype: int64

**Столбец 'gender '**

In [60]:
data['gender'].value_counts()

F      14154
M       7247
XNA        1
Name: gender, dtype: int64

Выявлено 1 некорректное значение - пол 'XNA', заменим его на 'F'. 

In [61]:
data.loc[data['gender']=='XNA','gender'] = 'F'
data['gender'].value_counts()

F    14155
M     7247
Name: gender, dtype: int64

**Столбец 'income_type'**

In [62]:
data['income_type'].value_counts()

сотрудник          11050
компаньон           5054
пенсионер           3839
госслужащий         1453
предприниматель        2
безработный            2
студент                1
в декрете              1
Name: income_type, dtype: int64

Без аномалий

**Столбец 'debt'**

In [63]:
data['debt'].value_counts()

0    19670
1     1732
Name: debt, dtype: int64

Без аномалий

**Столбец 'total_income'**

In [64]:
data['total_income'].describe()

count    1.924000e+04
mean     1.674488e+05
std      1.031051e+05
min      2.066726e+04
25%      1.030001e+05
50%      1.450208e+05
75%      2.034447e+05
max      2.265604e+06
Name: total_income, dtype: float64

Нужно избавиться от ненужных знаков после запятой.

**Столбец 'purpose'**

In [65]:
print(data['purpose'].value_counts())

свадьба                                   796
на проведение свадьбы                     772
сыграть свадьбу                           769
операции с недвижимостью                  673
покупка коммерческой недвижимости         661
покупка жилья для сдачи                   651
операции с жильем                         648
операции с коммерческой недвижимостью     646
жилье                                     642
покупка жилья                             641
покупка жилья для семьи                   640
недвижимость                              632
строительство собственной недвижимости    628
операции со своей недвижимостью           626
строительство жилой недвижимости          622
строительство недвижимости                620
покупка недвижимости                      619
покупка своего жилья                      619
ремонт жилью                              609
покупка жилой недвижимости                603
на покупку своего автомобиля              504
заняться высшим образованием      

Много взаимозаменяемых названий назначений кредита. 

### Шаг 2.3. Изменение типов данных.

In [66]:
total_income_median = data['total_income'].median()
print('Размер данных до преобразования:',data['total_income'].nbytes )
data['total_income'] = data['total_income'].fillna(total_income_median)
data['total_income'] = data['total_income'].astype(int)
print(data['total_income'].head(10))
data.info()
print('Размер данных после преобразования:',data['total_income'].nbytes )

Размер данных до преобразования: 171216
0    253875
1    112080
2    145885
3    267628
4    158616
5    255763
6    240525
7    135823
8     95856
9    144425
Name: total_income, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21402 entries, 0 to 21401
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21402 non-null  int64 
 1   dob_years         21302 non-null  object
 2   education         21402 non-null  object
 3   education_id      21402 non-null  int64 
 4   family_status     21402 non-null  object
 5   family_status_id  21402 non-null  int64 
 6   gender            21402 non-null  object
 7   income_type       21402 non-null  object
 8   debt              21402 non-null  int64 
 9   total_income      21402 non-null  int64 
 10  purpose           21402 non-null  object
dtypes: int64(5), object(6)
memory usage: 1.8+ MB
Размер данных после преобразования: 171216


### Шаг 2.4. Удаление дубликатов.

**Столбец 'education'**

Одинаковые значения в столбце 'education' написаны в разных регистрах. Приведём все значения к прописным с помощью str.lower(). 

In [67]:
data['education'] = data['education'].str.lower()
print(data['education'].value_counts())

среднее                15136
высшее                  5237
неоконченное высшее      741
начальное                282
ученая степень             6
Name: education, dtype: int64


Причиной возникновения дубликатов могло быть совмещение данных заполняемых разными людьми.

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

In [68]:
data_education = data[['education','education_id']]
data_education = data_education.drop_duplicates().reset_index(drop=True)
print(data_education.head(20))
data_family = data[['family_status','family_status_id']]
data_family = data_family.drop_duplicates().reset_index(drop=True)
print(data_family.head(20))

             education  education_id
0               высшее             0
1              среднее             1
2  неоконченное высшее             2
3            начальное             3
4       ученая степень             4
           family_status  family_status_id
0        женат / замужем                 0
1       гражданский брак                 1
2         вдовец / вдова                 2
3              в разводе                 3
4  не женат / не замужем                 4


In [69]:
data = data.drop(['education'],axis=1)
data.head()

,children,dob_years,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,1.0,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,1.0,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,0.0,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,3.0,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,0.0,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


### Шаг 2.6. Категоризация дохода.

In [70]:
def total_income_category(row):
    income = row['total_income']

    if income <= 30000:
        return 'E'
    if income > 30000 and income <= 50000:
        return 'D'
    if income > 50000 and income <= 200000:
        return 'C'
    if income > 200000 and income < 1000000:
        return 'B'
    if income > 1000000:
        return 'A'
data['total_income_category'] = data.apply(total_income_category, axis=1)
data.head()

,children,dob_years,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,1.0,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B
1,1,1.0,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,0.0,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C
3,3,3.0,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,0.0,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C


### Шаг 2.7. Категоризация целей кредита.

In [71]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21402 entries, 0 to 21401
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   children               21402 non-null  int64 
 1   dob_years              21302 non-null  object
 2   education_id           21402 non-null  int64 
 3   family_status          21402 non-null  object
 4   family_status_id       21402 non-null  int64 
 5   gender                 21402 non-null  object
 6   income_type            21402 non-null  object
 7   debt                   21402 non-null  int64 
 8   total_income           21402 non-null  int64 
 9   purpose                21402 non-null  object
 10  total_income_category  21402 non-null  object
dtypes: int64(5), object(6)
memory usage: 1.8+ MB


In [72]:
def purpose_category(purpose):
    for purpose_category in purpose:
        if 'автомобил' in purpose:
            return 'операции с автомобилем'
        if  ('недвиж' in purpose) or('жиль' in purpose):
            return 'операции с недвижимостью'
        if 'свадьб' in purpose:
            return 'проведение свадьбы'
        if 'образован' in purpose:
            return 'получение образования'
        return 'другое'
data['purpose_category'] = data['purpose'].apply(purpose_category)
data.head(50)

,children,dob_years,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,1.0,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,1.0,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,0.0,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,3.0,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,0.0,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы
5,0,0.0,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,B,операции с недвижимостью
6,0,0.0,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,B,операции с недвижимостью
7,0,0.0,1,женат / замужем,0,M,сотрудник,0,135823,образование,C,получение образования
8,2,2.0,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,C,проведение свадьбы
9,0,0.0,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,C,операции с недвижимостью


## Ответы на вопросы

### Узнаем, есть ли зависимость между количеством детей и возвратом кредита в срок

In [73]:
data_pivot_children = data.pivot_table(index=['debt'], columns='children', values = 'family_status_id', aggfunc='count')
print(data_pivot_children.head())
children_0 = data_pivot_children[0][1] / data_pivot_children[0][0] 
children_1 = data_pivot_children[1][1] / data_pivot_children[1][0] 
children_2 = data_pivot_children[2][1] / data_pivot_children[2][0]
children_3 = data_pivot_children[3][1] / data_pivot_children[3][0] 
children_4 = data_pivot_children[4][1] / data_pivot_children[4][0]
children_5 = data_pivot_children[5][1] / data_pivot_children[5][0]

print("{0:.2f}%   Нет детей".format(children_0*100))
print("{0:.2f}%  1 Ребенок".format(children_1*100))
print("{0:.2f}%  2 Ребенка".format(children_2*100))
print("{0:.2f}%   3 Ребенка".format(children_3*100))
print("{0:.2f}%  4 Ребенка".format(children_4*100))
print("{0:.2f}%    5 Детей".format(children_5*100))

children        0       1       2      3     4    5
debt                                               
0         13086.0  4374.0  1861.0  303.0  37.0  9.0
1          1063.0   444.0   194.0   27.0   4.0  NaN
8.12%   Нет детей
10.15%  1 Ребенок
10.42%  2 Ребенка
8.91%   3 Ребенка
10.81%  4 Ребенка
nan%    5 Детей


В основном при увеличении детей в семье растёт процент невыплаты. Исключение составляют семьи с 3-мя и 5-ю детьми. Для более точных утверждений надо знать количество детей в семье при выплате кредита.

### Узнаем, есть ли зависимость между семейным положением и возвратом кредита в срок

In [74]:
data_pivot_family = data.pivot_table(index=['debt'], columns='family_status',values='dob_years',  aggfunc='count')
print(data_pivot_family)
no_family = data_pivot_family['не женат / не замужем'][1] / data_pivot_family['не женат / не замужем'][0]
divorce = data_pivot_family['в разводе'][1] / data_pivot_family['в разводе'][0]
widow = data_pivot_family['вдовец / вдова'][1] / data_pivot_family['вдовец / вдова'][0]
partner = data_pivot_family['гражданский брак'][1] / data_pivot_family['гражданский брак'][0]
family = data_pivot_family['женат / замужем'][1] / data_pivot_family['женат / замужем'][0]
print('{0:.2f}% не женат / не замужем'.format(no_family*100))
print('{0:.2f}%  в разводе'.format(divorce*100))
print('{0:.2f}%  вдовец / вдова'.format(widow*100))
print('{0:.2f}% гражданский брак'.format(partner*100))
print('{0:.2f}%  женат / замужем'.format(family*100))

family_status  в разводе  вдовец / вдова  гражданский брак  женат / замужем  \
debt                                                                          
0                   1095             885              3756            11331   
1                     84              62               383              923   

family_status  не женат / не замужем  
debt                                  
0                               2511  
1                                272  
10.83% не женат / не замужем
7.67%  в разводе
7.01%  вдовец / вдова
10.20% гражданский брак
8.15%  женат / замужем


Самый большой процент просрочек у не женатых и незарегистрировавших свой брак. Значительно реже просрочки встречаются у женатых и разведёных. Можно предположить, что семейные пары серьезней относятся к кредитным обязанностям, а разведёные при расставании разделяют обязательства. Самые ответственные заёмщики - вдовцы. Как и в предыдущем вопросе, мы не знаем семейное положение на конец обслуживания кредита.

### Узнаем, есть ли зависимость между уровнем дохода и возвратом кредита в срок

In [75]:
data_pivot_income_rating = data.groupby(['total_income_category']).agg({'debt':['sum', 'count']})
data_pivot_income_rating[('debt', 'sum/count, %')] = data_pivot_income_rating[('debt', 'sum')] / data_pivot_income_rating[('debt', 'count')]*100
print(data_pivot_income_rating)

                       debt                    
                        sum  count sum/count, %
total_income_category                          
A                         2     25     8.000000
B                       354   5013     7.061640
C                      1353  15993     8.459951
D                        21    349     6.017192
E                         2     22     9.090909


Больше всех кредитуется категория людей со средним уровнем дохода и показатель невозврата кредита в срок там наиболее большой. Люди с минимальным доходом лучше обслуживают кредит, чем клиенты со сверхвысоким. Самые ответственные клиенты с высоким доходом из группы "B". 

### Узнаем, как разные цели кредита влияют на его возврат в срок

In [76]:
data_pivot_purpose = data.groupby(['purpose_category']).agg({'debt':['sum', 'count']})
data_pivot_purpose[('debt', 'part,%')] = data_pivot_purpose[('debt', 'sum')] / data_pivot_purpose[('debt', 'count')]*100
print(data_pivot_purpose)

                         debt                 
                          sum  count    part,%
purpose_category                              
операции с автомобилем    400   4288  9.328358
операции с недвижимостью  780  10780  7.235622
получение образования     369   3997  9.231924
проведение свадьбы        183   2337  7.830552


В процентном соотношении минимальная доля просрочек у приобретателей недвижимости, которая является залогом, что дисциплинирует. Молодожены также серьёзно относятся к кредитам.  

Обслуживание категорий "автомобиль" и "образование" приносят банкам больше проблем.

## Общий вывод:

На основании входных данных от банка можно сделать вывод, что семейное положение, количество детей и уровень заработной платы значительно влияют на своевременный возврат кредита. Наиболее привлекательными для банков являются клиенты без детей, находящиеся в официальных отношениях, и приобретающие недвижимость. Наименее привлекательными являются клиенты с 1-2 детьми, находящиеся в  незарегистрированных отношениях, с целью кредита - свадьба или образование. 